In [7]:
#matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os

import importlib
import FVMF
importlib.reload(FVMF)
import test_ensemble
importlib.reload(test_ensemble)

prefix = "_phoneme_bg_"
# define the summary writer
writer = SummaryWriter()
sns.set()
sns.set_style("dark")
sns.set_palette("muted")
sns.set_color_codes("muted")


# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

if (torch.cuda.is_available()):
    print("GPUs are used!")
else:
    print("CPUs are used!")

# define the parameters
BATCH_SIZE = 100
TEST_BATCH_SIZE = 100
COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50

#prepare the data
data = pd.read_csv('http://www.uio.no/studier/emner/matnat/math/STK2100/data/phoneme.data')
data = data.drop(columns=["row.names"])
data = pd.concat([data,data.g.astype("category").cat.codes.astype(int)],sort=False, axis=1) #get_dummies(data['g'], prefix='phoneme')],sort=False, axis=1)
data = data.drop(columns=["g","speaker"])
data = data.values

np.random.seed(40590)

tr_ids = np.random.choice(4509, 3500, replace = False)
te_ids = np.setdiff1d(np.arange(4509),tr_ids)[0:1000]

dtrain = data[tr_ids,:]

data_mean = dtrain.mean(axis=0)[0:256]
data_std = dtrain.std(axis=0)[0:256]

data[:,0:256] = (data[:,0:256]  - data_mean)/data_std

dtrain = data[tr_ids,:]
dtest = data[te_ids,:]


# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])


#The net does not like to get larger at a given layer??
l1shape=(256, 10)
l2shape=(10, 10)
l3shape=(10, 10)
l4shape=(10, 5)
layershapes = [l1shape, l2shape, l3shape, l4shape]

epochs = 10
trtimes  = np.zeros(epochs)
# make inference on 10 networks
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net = FVMF.BayesianNetwork(layershapes=layershapes,BN='notbatchnorm',VD='Gaussian',
                               dtrain=dtrain,dtest=dtest,BATCH_SIZE = 100,classification = 'classification').to(DEVICE)
    #net = VMF.BayesianNetwork(l1=l1shape, l2=l2shape, l3=l3shape,l4=l4shape,BN='notbatchnorm').to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr=0.007)
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,shape = (0,256,256,257))
        #print(net.l1.weight_mu.mean())

    res = test_ensemble.test_ensemble(net,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = (0,256,256,257))

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

GPUs are used!
FVMF RELOADED
GPUs are used!
0
Random Init Utilized


/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
loss: tensor(350.0442, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(49.1581, device='cuda:1', grad_fn=<NllLossBackward0>)
2
loss: tensor(297.2805, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(16.6147, device='cuda:1', grad_fn=<NllLossBackward0>)
3
loss: tensor(276.6905, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(13.7142, device='cuda:1', grad_fn=<NllLossBackward0>)
4
loss: tensor(256.4194, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(12.5797, device='cuda:1', grad_fn=<NllLossBackward0>)
5
loss: tensor(239.7066, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(13.1582, device='cuda:1', grad_fn=<NllLossBackward0>)
6
loss: tensor(219.7606, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(12.9421, device='cuda:1', grad_fn=<NllLossBackward0>)
7
loss: tensor(201.7312, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihoo

In [2]:
w_mu_nodewise = []
b_mu = []
for i,layer in enumerate(net.layers):
    w_mu_nodewise += [layer.weight_mu]
    b_mu += [layer.bias_mu]

In [3]:
#import importlib
#import os
import FVMF
importlib.reload(FVMF)

#import VMF

#import importlib
#importlib.reload(VMF)

prefix = "_phoneme_bg_"
# define the summary writer
writer = SummaryWriter()
sns.set()
sns.set_style("dark")
sns.set_palette("muted")
sns.set_color_codes("muted")


# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

# define the parameters
BATCH_SIZE = 100
TEST_BATCH_SIZE = 100
batch_size = 100
COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50

#prepare the data
data = pd.read_csv('http://www.uio.no/studier/emner/matnat/math/STK2100/data/phoneme.data')
data = data.drop(columns=["row.names"])
data = pd.concat([data,data.g.astype("category").cat.codes.astype(int)],sort=False, axis=1) #get_dummies(data['g'], prefix='phoneme')],sort=False, axis=1)
data = data.drop(columns=["g","speaker"])
data = data.values

np.random.seed(40590)

tr_ids = np.random.choice(4509, 3500, replace = False)
te_ids = np.setdiff1d(np.arange(4509),tr_ids)[0:1000]

dtrain = data[tr_ids,:]

data_mean = dtrain.mean(axis=0)[0:256]
data_std = dtrain.std(axis=0)[0:256]

data[:,0:256] = (data[:,0:256]  - data_mean)/data_std

dtrain = data[tr_ids,:]
dtest = data[te_ids,:]

# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])

epochs = 10
trtimes  = np.zeros(epochs)
#w_mu = [w_mu1, w_mu2, w_mu3, w_mu4]
#b_mu = [b_mu1, b_mu2, b_mu3, b_mu4]

#w_mu_nodewise = [w_mu1_nodewise,w_mu2_nodewise,w_mu3_nodewise,w_mu4_nodewise]
#b_mu_nodewise = [b_mu1_nodewise,b_mu2_nodewise,b_mu3_nodewise,b_mu4_nodewise]
# make inference on 10 networks

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net2 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(4,4.1),
                                w_kappa=torch.Tensor(1).uniform_(6.5,6.6),
                                Temper = 1, normalize = 'No',classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #p.requires_grad_(False)
    
    #for j,p in enumerate(net2.l2.parameters()):
    #p.requires_grad_(False)
    
    optimizer = optim.Adam(net2.parameters(), lr=0.14)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net2, dtrain, SAMPLES, optimizer, epoch, i,shape = (0,256,256,257),BATCH_SIZE = 100)
        print('max:',net2.weight_mu[1].max())
        print('norm:',torch.norm(net2.weight_mu[1]))

    res = test_ensemble.test_ensemble(net2,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = (0,256,256,257))

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

GPUs are used!
FVMF RELOADED
0
Random Init Utilized


/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
loss: tensor(208.9301, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(53.8483, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(4.3046, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(15.0372, device='cuda:1', grad_fn=<NormBackward1>)
2
loss: tensor(198.2865, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(17.0525, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(4.9646, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(17.5934, device='cuda:1', grad_fn=<NormBackward1>)
3
loss: tensor(169.0435, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(19.6990, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(5.8137, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(18.7951, device='cuda:1', grad_fn=<NormBackward1>)
4
loss: tensor(159.2988, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(15.4887, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(6.060

KeyboardInterrupt: 

In [3]:
#matplotlib inline
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os

import importlib
import FVMF
importlib.reload(FVMF)
import test_ensemble
importlib.reload(test_ensemble)

prefix = "_phoneme_bg_"
# define the summary writer
writer = SummaryWriter()
sns.set()
sns.set_style("dark")
sns.set_palette("muted")
sns.set_color_codes("muted")


# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

if (torch.cuda.is_available()):
    print("GPUs are used!")
else:
    print("CPUs are used!")

# define the parameters
BATCH_SIZE = 100
TEST_BATCH_SIZE = 100
COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50

#prepare the data
data = pd.read_csv('http://www.uio.no/studier/emner/matnat/math/STK2100/data/phoneme.data')
data = data.drop(columns=["row.names"])
data = pd.concat([data,data.g.astype("category").cat.codes.astype(int)],sort=False, axis=1) #get_dummies(data['g'], prefix='phoneme')],sort=False, axis=1)
data = data.drop(columns=["g","speaker"])
data = data.values

np.random.seed(40590)

tr_ids = np.random.choice(4509, 3500, replace = False)
te_ids = np.setdiff1d(np.arange(4509),tr_ids)[0:1000]

dtrain = data[tr_ids,:]

data_mean = dtrain.mean(axis=0)[0:256]
data_std = dtrain.std(axis=0)[0:256]

data[:,0:256] = (data[:,0:256]  - data_mean)/data_std

dtrain = data[tr_ids,:]
dtest = data[te_ids,:]


# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])


#The net does not like to get larger at a given layer??
l1shape=(256, 10)
l2shape=(10, 10)
l3shape=(10, 10)
l4shape=(10, 5)
layershapes = [l1shape, l2shape, l3shape, l4shape]

epochs = 10
trtimes  = np.zeros(epochs)
# make inference on 10 networks
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net = FVMF.BayesianNetwork(layershapes=layershapes,BN='notbatchnorm',VD='ProjGaus',
                               dtrain=dtrain,dtest=dtest,BATCH_SIZE = 100,classification = 'classification').to(DEVICE)
    #net = VMF.BayesianNetwork(l1=l1shape, l2=l2shape, l3=l3shape,l4=l4shape,BN='notbatchnorm').to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr=0.07)
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,shape = (0,256,256,257))
        #print(net.l1.weight_mu.mean())
        print('max:',net.weight_mu[1].max())
        print('norm:',torch.norm(net.weight_mu[1]))
    res = test_ensemble.test_ensemble(net,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = (0,256,256,257))

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

GPUs are used!
FVMF RELOADED
GPUs are used!
0
Random Init Utilized


/opt/uio/modules/rhel8/easybuild/software/Miniconda3/lmsunde/envs/BNN2/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
loss: tensor(-36.8351, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(171.1453, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(1.3029, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(6.3531, device='cuda:1', grad_fn=<NormBackward1>)
2
loss: tensor(-72.8562, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(160.2231, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(1.3919, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(7.0493, device='cuda:1', grad_fn=<NormBackward1>)
3
loss: tensor(-90.4584, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(159.4871, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(1.5872, device='cuda:1', grad_fn=<MaxBackward1>)
norm: tensor(7.5766, device='cuda:1', grad_fn=<NormBackward1>)
4
loss: tensor(-103.7574, device='cuda:1', grad_fn=<AddBackward0>)
negative_log_likelihood: tensor(159.0381, device='cuda:1', grad_fn=<NllLossBackward0>)
max: tensor(1.6

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42069)
torch.manual_seed(42069)

PL = [torch.Tensor([1-3, 1-3]),torch.Tensor([5-3, 1-3]),torch.Tensor([1-3, 5-3]),torch.Tensor([5-3, 5-3]),torch.Tensor([3-3, 3-3])]
cov = torch.eye(2)
n = 250

plt.figure(figsize=(10, 10), dpi=500)
distrib = []

DF = torch.zeros((len(PL),n,1,3)) #multiple of 4
for i, MU in enumerate(PL): #enumerate starts from and including 0.
    distrib.append(torch.distributions.multivariate_normal.MultivariateNormal(loc=MU, covariance_matrix=cov))
    DATA_ = distrib[i].sample((n,1))

    DATA  = torch.zeros([n, 1, 3])
    DATA[:,:,:2] = DATA_
    DATA[:,:,2]  = i
    DF[i,:,:,:] = DATA
    
    x = DATA[:,0,0]
    y = DATA[:,0,1]
    plt.plot(x,y,'.',markersize=1.25)
plt.show()

C = int(3*n/4)

#DATA_train = torch.zeros((len(PL)*C,3))
#DATA_test = torch.zeros((len(PL)*(n-C),3))

DATA = DF.reshape(len(PL)*n,3)
#print('DATA:',DATA,'len(DATA):',len(DATA),'mean dtrain:',DATA.mean(axis=0)[2])

#data_mean = DATA.mean(axis=1)[0:2]
#data_std = DATA.std(axis=1)[0:2]

#DATA[:,0:2] = (DATA[:,0:2]  - data_mean)/data_std
#print('DATA normalized:',DATA,'len(DATA) normalized:',len(DATA),'mean dtrain normalized:',DATA.mean(axis=0)[2])
tr_ids = np.random.choice(n*5, n*4, replace = False)



dtrain = DATA[tr_ids,:]
dtest = DATA[-tr_ids,:]

#print('\n','dtrain:',dtrain, 'len(dtrain):',len(dtrain),'mean dtrain:',dtrain.mean(axis=0)[2])
#print('\n','dtest:',dtest, 'len(dtest):',len(dtest),'mean dtest:',dtest.mean(axis=0)[2])

In [ ]:
import math
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import importlib


import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

# define the summary writer
writer = SummaryWriter()
# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

# define the parameters

COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50
TEST_BATCH_SIZE = 100
BATCH_SIZE = 100

l1shape=(2, 5)
l2shape=(5, 5)
l3shape=(5, 5)
l4shape=(5, 5)
l5shape=(5, 5)
l6shape=(5, 5)
layershapes = [l1shape, l2shape, l3shape, l4shape]

# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])
data_shape = (0,2,2,3)

epochs = 30
trtimes  = np.zeros(epochs)
#w_mu = [w_mu1, w_mu2, w_mu3, w_mu4]
#b_mu = [b_mu1, b_mu2, b_mu3, b_mu4]

#w_mu_nodewise = [w_mu1_nodewise,w_mu2_nodewise,w_mu3_nodewise,w_mu4_nodewise]
#b_mu_nodewise = [b_mu1_nodewise,b_mu2_nodewise,b_mu3_nodewise,b_mu4_nodewise]
# make inference on 10 networks

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net3 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(4,4.1),
                                #w_kappa=torch.Tensor(1).uniform_(6,6.1),
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net3.parameters(), lr=0.04)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net3, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        print('max:',net3.weight_mu[1].max())
        print('norm:',torch.norm(net3.weight_mu[1]))

    res = test_ensemble.test_ensemble(net3,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

In [ ]:
import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

epochs = 10

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(4.0,4.1),
                                w_kappa=torch.Tensor(1).uniform_(6.0,6.1),
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4.parameters(), lr=0.14)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    #res = test_ensemble.test_ensemble(net4,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

In [ ]:
GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag.

w_mu = []
for i in range(len(net4.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(net4.weight_mu[i]/torch.norm(net4.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu = []
for i in range(len(net4.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(net4.bias_mu[i]/torch.norm(net4.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho= []
for i in range(len(net4.bias_rho)):
    b_rho.append(net4.bias_rho[i]*GC_intensity)

w_rho= []
for i in range(len(net4.weight_rho)):
    w_rho.append(net4.weight_rho[i]*GC_intensity)

#print('b_rho:',b_rho, '\n','w_rho:',b_mu)

In [ ]:
epochs = 10
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4_GC = FVMF.BayesianNetwork(w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4_GC.parameters(), lr=0.07)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4_GC, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    #res = test_ensemble.test_ensemble(net4_GC,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
#print('This is the loss with the Gradient Capture method/Normalized Initialization Inheritance')

#Ok, så kanskje drit i gradient-capture-method. Men! Husk hva du har vist her nå med kun normalisering av vektene, Du får trent nettet
#Videre, og det fungerer faktisk. I tilfellet hvor du ikke gjør det ender du faktisk ikke bare opp med dårlig ytelse, men
#hele nettet brekker!!!!!!!!!!!

In [ ]:
GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag. GC fungerer rett og slett ikke.

w_mu = []
for i in range(len(net4_GC.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(net4_GC.weight_mu[i]/torch.norm(net4_GC.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu = []
for i in range(len(net4_GC.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(net4_GC.bias_mu[i]/torch.norm(net4_GC.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho= []
for i in range(len(net4_GC.bias_rho)):
    b_rho.append(net4_GC.bias_rho[i]*GC_intensity)

w_rho= []
for i in range(len(net4_GC.weight_rho)):
    w_rho.append(net4_GC.weight_rho[i]*GC_intensity)

#print('b_rho:',b_rho, '\n','w_rho:',b_mu)

In [ ]:
epochs = 10
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4_GC2 = FVMF.BayesianNetwork(w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4_GC2.parameters(), lr=0.035)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4_GC2, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    res = test_ensemble.test_ensemble(net4_GC2,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
print('This is the loss with the Gradient Capture method/Normalized Initialization Inheritance, applied twice')

#Ok, så kanskje drit i gradient-capture-method. Men! Husk hva du har vist her nå med kun normalisering av vektene, Du får trent nettet
#Videre, og det fungerer faktisk. I tilfellet hvor du ikke gjør det ender du faktisk ikke bare opp med dårlig ytelse, men
#hele nettet brekker!!!!!!!!!!!

In [ ]:
RES = 1028 #Ca. Full HD resolution. If you struggle with memory adjust this parameter down. 64 for example already gives a decent view.
enums = 10

DEVICE = torch.device("cuda:1")
Temps = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net4_GC2(Temps[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(torch.max(Temps[:,2]),torch.min(Temps[:,2]))

In [ ]:
plt.title('vMF Uncertainty Plot 42epocsLR0.1 3*10epocs, NII in between.')
#xy = [0,0.2,0.4,0.6,0.8,1]
#z = xy
plot = plt.scatter(Temps[:,0].detach().numpy(), Temps[:,1].detach().numpy(),vmin = 0, vmax = 1.4, s=221000/(RES**2), c=Temps[:,2].detach().numpy(), cmap='RdYlBu', marker='s')
plt.gcf().set_size_inches((14, 12))
plt.colorbar(plot)
plt.show()

In [ ]:
RES = 1028
enums = 10

DEVICE = torch.device("cuda:1")
Temps2 = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps2[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net3(Temps2[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps2[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(probs.shape)

In [ ]:
plt.title('Gaussian Uncertainty Plot')
#xy = [0,0.2,0.4,0.6,0.8,1]
#z = xy
plot2 = plt.scatter(Temps2[:,0].detach().numpy(), Temps2[:,1].detach().numpy(),vmin = 0, vmax = 1.4, s=221000/(RES**2), c=Temps2[:,2].detach().numpy(), cmap='RdYlBu', marker='s')
plt.gcf().set_size_inches((14, 12))
plt.colorbar(plot2)
plt.show()

In [ ]:
#All of the variables should be defined earlier, to be certain we are comparing the same things in our plots.
RES = 1028
DEVICE = torch.device("cuda:1")
Temps4 = torch.zeros((RES**2,3))

x = torch.linspace(-5,5,RES)
Temps4[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((RES**2, 5)).to(DEVICE)


#print(distributions[i].log_prob(Temps4[:,0:2]).shape)
sfm = torch.nn.Softmax(dim = -1)

VAL = torch.zeros((RES**2,5)).to(DEVICE)
for i in range(len(distrib)):
    VAL[:,i] = distrib[i].log_prob(Temps4[:,0:2]) #Does the log_prob method really do exactly what I want?

VAL2 = torch.zeros((RES**2,5)).to(DEVICE)
VAL2 = sfm(VAL)
#mc = torch.min(VAL,dim = 1).values    
#nc = torch.sum(VAL-mc,dim = 1)

      
#for i in range(len(distrib)):
#    VAL[:,i] =  VAL[:,i]-mc)/nc[i]
    
Temps4[:,2] = -torch.sum(torch.log(VAL2) * VAL2,dim = 1) #We don't log the probs here, since they were already logged.


#print(VAL.shape)
#print(torch.exp(net3(Temps2[:,0:2])).shape)
print(max(torch.exp(distrib[i].log_prob(Temps4[:,0:2]))),min(torch.exp(distrib[i].log_prob(Temps4[:,0:2]))))

In [ ]:
plt.title('True Uncertainty Plot')
#xy = [0,0.2,0.4,0.6,0.8,1]
#z = xy
plot4 = plt.scatter(Temps4[:,0].detach().numpy(), Temps4[:,1].detach().numpy(), s=221000/(RES**2), c=Temps4[:,2].detach().numpy(), cmap='RdYlBu', marker='s')
plt.gcf().set_size_inches((14, 12))
plt.colorbar(plot4)
plt.show()

In [ ]:
#net4.state_dict()
#print(net4.state_dict()['layers.3.weight_mu'])
#get a bar for the range of the entropy scalar.
#Center data around 0.

#Simuler marginalfordeling for en vekt fra vMF. OG sammenlign med en marginal vekt fra multivariat Gaussisk som har blitt normalisert.
#og hva skjer i posterioren etter det er trent. trekk vekter fra HU, gjør forwardpass predictions som simulert data, tren på det og sjekk om vektene blir som i starten.
#Hva med å initialisere fra HU!!!???

#Siden W_kappa initialiseres så høyt, er det jo ikke så rart at vMF-en generelt er mye mindre selvsikker enn den Gaussiske.
#Da er det også forståelig, at ettersom vi øker w_kappa, så må vi også øke læringsraten, siden forandringen i likelihood
#loss ved weight_mu justering blir mindre intens. Kanskje er også noe av grunnen til at vi blir tvunget til så høye kappa,
#At nettet ikke klarer å finne noen gradient descent ellers? Siden hvis vi er langt unna der W-mu skulle vært, så får vi nesten
#ikke noen forandring i loss? Hva med en tostegs-trening, hvor vi først trener med høy kappa. Og så initialiserer vi fra de foregående
#W og b muene, men med nye tilfeldige kappa som er mye mindre?? Det høres ut som en god ide!

In [ ]:
import pandas as pd
import numpy as np
import torch
DATA = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00448/carbon_nanotubes.csv',sep = ';', decimal=",")
#This is the carbon nanotubes data.
DATA = DATA.astype(float)
#print(DATA)
#print(DATA)
DATA = DATA.to_numpy()
DATA = torch.from_numpy(DATA)
#print(DATA)
np.random.seed(42069)
#torch.manual_seed(42069)

DATA = DATA.type(torch.float32)

data_mean = DATA.mean(axis=1)[0:8]
data_std = DATA.std(axis=1)[0:8]
DEVICE = torch.device("cuda:1")

#DATA[:,7] = 5*DATA[:,1] + 3*DATA[:,6] + 2*DATA[:,3] + 1*DATA[:,2] 
DATA[:,0:8] = (DATA[:,0:8]  - data_mean)/data_std
#print('DATA normalized:',DATA,'len(DATA) normalized:',len(DATA),'mean dtrain normalized:',DATA.mean(axis=0)[2])
tr_ids = np.random.choice(10721, 6000, replace = False)

dTRAIN_CARBON = DATA[tr_ids,:]
dTEST_CARBON = DATA[-tr_ids,:]

print(DATA)

In [ ]:
import math
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import importlib

COND_OPT = False
CLASSES = 1
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 30
epochs =10
TEST_BATCH_SIZE = 6000
BATCH_SIZE = 100

import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

data_shape = (0,7,7,8)
l1shape=(7, 7)
l2shape=(7, 7)
l3shape=(7, 7)
l4shape=(7, 1)
layershapes = [l1shape, l2shape, l3shape, l4shape]

trtimes  = np.zeros(epochs)


for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net5= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                #w_kappa=torch.Tensor(1).uniform_(5,5.1),
                                Temper = 0.1,
                                classification='Regression')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net5.parameters(), lr=0.0007)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net5, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(net5,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

In [ ]:
import math
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import importlib

COND_OPT = False
CLASSES = 1
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 30
epochs = 10
TEST_BATCH_SIZE = 6000
BATCH_SIZE = 100

import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

data_shape = (0,7,7,8)
l1shape=(7, 7)
l2shape=(7, 7)
l3shape=(7, 7)
l4shape=(7, 1) #So the vMF mathematically does not support having a layer that ends in 1. This is the cause of the bug.
#However, should we even have the last layer as a vmf for regression? I don't really think so. Lets try the layerwise vmf
#for the last one. Wait... why is the layerwise x_dim=out*in for its weights while the nodewise is just x_dim=in? Yeah cause I
#list stuff together inside the vmf_nodewise, so that it makes sense. Maybe it will work better to just have a frequenstist last layer?
#The loss is not calculated correctly, MSE criterion 0.92, is not representative of the network missing by 42000 on a 0-1 regression task.
#There was some PyTorch funny business with the lr being so low that the los becomes jumpy and the pedictions rubbish.... No
#immediate mathematical interpretation of this strange behaviour as far as I can see, so I think it is a technical probelm
#with regard to loss or gradient calculation in the PyTorch machinery.

#But bro if the out_features are 1, then in=out*in! So the layerwise vmf won't help you. Let's do Gaussian.
layershapes = [l1shape, l2shape, l3shape, l4shape]

trtimes  = np.zeros(epochs)


#Note, for this regression task, the last 7 to 1 layer has a Gaussian VD, where we kill the prior and simply optimize with MLE.
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    vMFRegression= FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(3,3.1),
                                w_kappa=torch.Tensor(1).uniform_(7.5,7.6),
                                Temper = 0,classification='Regression',NODEFORCE =False)
    
    #for j,p in enumerate(net2.l1.parameters()):
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(vMFRegression.parameters(), lr=0.11)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(vMFRegression, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

In [ ]:
GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag. GC fungerer rett og slett ikke.

w_mu_R = []
for i in range(len(vMFRegression.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(vMFRegression.weight_mu[i]/torch.norm(vMFRegression.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu_R = []
for i in range(len(vMFRegression.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(vMFRegression.bias_mu[i]/torch.norm(vMFRegression.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    

In [ ]:

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    vMFRegression2= FVMF.BayesianNetwork(w_mu = w_mu_R, b_mu = b_mu_R, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dTRAIN_CARBON, dtest=dTEST_CARBON,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'regression')
    
    #for j,p in enumerate(net2.l1.parameters()):
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(vMFRegression2.parameters(), lr=0.025)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(vMFRegression2, dTRAIN_CARBON, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100,
                                    shape = data_shape,CLASSES = 1)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    loss, outputs, output = test_ensemble.test_ensemble(vMFRegression2,dTEST_CARBON,TEST_SAMPLES,TEST_BATCH_SIZE,TEST_BATCH_SIZE,
                                      CLASSES,DEVICE,shape = data_shape,classification = False,plot = True)

In [ ]:
g = [0,1,2,3,4,5]
for i,G, in enumerate(g):
    print('i:',i,'g:',G)
print(len(g))
i,j = True,True
if (i==1) or (j==1):
    print('The or statement in Python means and/or')
else:
    print('The or statement in Python means either or')

In [ ]:
#The RSS doesn't really vary much based on the number of epochs I administer. I suspect this to be either because we are not
#training the network correctly, or we are testing it incorrectly.

#Make many forward passes, and calculate the MSE on all of the forward passes!

#Also calculate the percentiles for them!

import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import pandas as pd
import torchvision.datasets as datasets


import os

cifar_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=None)
cifar_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=None)

In [ ]:
#print(cifar_train.__getitem__(1)[0][1])
#print(cifar_test.__getitem__(1))

In [ ]:
r"""
What if the problem is that since the mu's in the Gaussian is (out,in), and the vMF is out*in,
this could mean that we have "strethed" a single vMF pdf over all the parameters, while in the Gaussian we have made one for each output?

I don't know, also intuitively this should be the case, since the whole point of the vMF is the norm 1, which obviously
will require that it for each forward-pass is only one massive pdf for all inputs and outputs.

Ok, this can get rough, I will try my best to reshape the w_mu's and b_mu's the best I can, perhaps this will work just fine without too much tuning...
Hopefully.

This doe not seem to ave worked. I don't konw exactly what the problem is. Per haps it is a good idea to consult the new loss function
suggested in the paper that made the code we based our vMF on?

It is very strange that the loss is not at all affected by completely ridiculous learning rates...

Remember that vMF makes the norm of the weights and biases 1, not the forward pass of the x's. Hence the advantage is that the gradient
will not explode, since the backward pass of it will also be approx. 1. In batchnorm, maybe the gradient can explode? Since the weights 
can be whatever?


IMPORTANT: The gaussian neuralnet will also collapse to 257 if I apply more than 3 layers. This must be somehow related to the similar
behavior in the vMF when the size of each layer exceeds 3. Per haps there is an error in the loss afterall?
However, in the Gaussian case, increasing the learning rate by a factor of 10 solved the issue. This makes me suspect it is the mathematical
properties of the loss function, rather than incorrect implementation.
"""

#When l4 is (3,5):
r"""
File ~/projects/BNN/AliaksandrFolder/FVMF.py:289, in vMF.sample(self, N, rsf)
    287 e1mu = torch.zeros(d, 1).to(DEVICE)
    288 e1mu[0, 0] = 1.0
--> 289 e1mu = e1mu - self.mu if len(self.mu.shape) == 2 else e1mu - self.mu.unsqueeze(1) #e1mu.shape = (1,self.x_dim). mu_unnorm.shape = (mu_unnorm)
    290 e1mu = e1mu / norm(e1mu, dim=0).to(DEVICE)
    291 samples = samples - 2 * (samples @ e1mu) @ e1mu.t()

RuntimeError: The size of tensor a (15) must match the size of tensor b (9) at non-singleton dimension 0
"""

#When l4 is (5,5):
r"""
File ~/projects/BNN/AliaksandrFolder/FVMF.py:289, in vMF.sample(self, N, rsf)
    287 e1mu = torch.zeros(d, 1).to(DEVICE)
    288 e1mu[0, 0] = 1.0
--> 289 e1mu = e1mu - self.mu if len(self.mu.shape) == 2 else e1mu - self.mu.unsqueeze(1) #e1mu.shape = (1,self.x_dim). mu_unnorm.shape = (mu_unnorm)
    290 e1mu = e1mu / norm(e1mu, dim=0).to(DEVICE)
    291 samples = samples - 2 * (samples @ e1mu) @ e1mu.t()

RuntimeError: The size of tensor a (25) must match the size of tensor b (15) at non-singleton dimension 0
"""
#These errors above were caused by my initialization being wrong. I copy paster mu_3 for layer4, and forgot to change to mu_4. So now 
#I always get the error below.


#in all cases now: 

r"""
It seems that the whole thing does not progress at all. We just get the warning and then no further output.

self.l4(x, sample)

--> self.bias.sample()

It always get's stuck there!!

Specifically, it get's stuck in the while loop:

while len(v0) < N:
            eps = beta.sample([1, rsf * (N - len(v0))]).squeeze().to(DEVICE)
            uns = uniform.sample([1, rsf * (N - len(v0))]).squeeze().to(DEVICE)
            w0 = (1 - (1 + bb) * eps) / (1 - (1 - bb) * eps)
            t0 = (2 * aa * bb) / (1 - (1 - bb) * eps)
            det = (d - 1) * t0.log() - t0 + dd - uns.log()
            v0 = torch.cat([v0, torch.tensor(w0[det >= 0]).to(DEVICE)])
            if len(v0) > N:
                v0 = v0[:N]
                break
"""

r"""
From further investigations it is clear that the error lies in w0[det >= 0] consistently being an empty Tensor.

Even further, bb is 0 here which it usually is not. That must definitely indicate something is wrong.

Adjusting the initialization of kappa to be 9 or less on both weights and biases makes the code run, 
but posterior collapse is back. Increasing kappa seems to increase the compute aswell... however, getting the kappa inits
closer to 10 seems to also help avoid the posterior collapse. And the lower bound increased also helps, looks like 3 is optimal.
"""


r"""
It seems that Torch likes that each of the bias_mu's and weight_mu's from each layer are separately registered with name as an nn.parameter.
"""

r"""
Currently, for 4 layers with 3 hiddenwidth, it seems around 10 epochs with .14 learning rate is optimal for testperformance.
I suspect this is because we do not have the modelcapacity to go beyond the overfiting case just yet, and must settle for the classical
best-fit.
"""

r"""
The problem for both the random initialization and the variable length Gaussian has the same root. It is that the mu's and rho's 
are not being registered as parameters per layer to begin with!

And since this registering works just fine when we are directly assigning self.layer's to be each layer the parameters are registered correctly,
it must be the case that this part in the BayesianNetwork's initializer is where the problem originates.
"""

r"""
Part of the reason why the layerwise vMF might not be that performant, 
is that it forces one kappa on every weight in the entire layer.
"""

r"""
The way the mu of the weights are used in the vMF is that any mu is accepted, and then the mu's are normalized. 
I think this is the reason for why the net becomes intractably slow as we increase the layers. 
If it is possible to set the weight_mu parameters to be the normalized versions for every epoch I think that would be great.

Or perhaps build into the loss a term that penalizes the weights from deviating from norm=1...

I don't know exactly where to put this
"""

r"""
The training function I got from Aliaksandr uses Variable() which is depcrecated since PyTorch 0.4, and creates tensors directly 
via torch.FloatTensor which is also bad practice. In addition the data is presented to the train function as numpy arrays.

All these three issues require resolution.

I starred a S.Overflow post that showed what I think will be the solution to the ghost-Mu. Remember that you could motivate this 
mathematically by proving dF(x/norm(x);parameters)/dx = dF(x;parameters)/dnorm(x)

Should I try to see what happens if I use float64 instead?
"""

r"""
Should I really have a vMF on the biases? They don't contribute to the gradient much in the backward-pass, 
making them norm 1 doesn't really make too much sense..
"""

r"""
[Kappa is called concentration parameter because the higher the kappa the lower the variance, 
i.e. the higher the concentration around mu. This also explains why the vMF struggles for higher kappas, 
because it is known that the vMF is numerically heavy and unstable for high concentrations, aka low variance.
"""

In [ ]:
r"""
Plan:

First solve the uncertainty estimation.

Solve comparing deeper nets, with alternating use of batchnorm.
Solve a couple more image-classification data sets
Solve the nn.Parameter(the Mu's) normalization business.
Solve having separate kappas for each dimension of the vMF. (I probably won't get to this..)

Compare with FNN's?
"""

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42069)
torch.manual_seed(42069)

PL = [torch.Tensor([1-3, 1-3]),torch.Tensor([5-3, 1-3]),torch.Tensor([1-3, 5-3]),torch.Tensor([5-3, 5-3]),torch.Tensor([3-3, 3-3])]
cov = torch.eye(2)
n = 150

plt.figure(figsize=(10, 10), dpi=500)

DF = torch.zeros((len(PL),n,1,3)) #multiple of 4
for i, MU in enumerate(PL): #enumerate starts from and including 0.
    distrib = torch.distributions.MultivariateNormal(loc=MU, covariance_matrix=cov)
    DATA_ = distrib.sample((n,1))

    DATA  = torch.zeros([n, 1, 3])
    DATA[:,:,:2] = DATA_
    DATA[:,:,2]  = i
    DF[i,:,:,:] = DATA
    
    x = DATA[:,0,0]
    y = DATA[:,0,1]
    plt.plot(x,y,'.',markersize=1.25)
plt.show()

C = int(3*n/4)

#DATA_train = torch.zeros((len(PL)*C,3))
#DATA_test = torch.zeros((len(PL)*(n-C),3))

DATA = DF.reshape(len(PL)*n,3)
#print('DATA:',DATA,'len(DATA):',len(DATA),'mean dtrain:',DATA.mean(axis=0)[2])

#data_mean = DATA.mean(axis=1)[0:2]
#data_std = DATA.std(axis=1)[0:2]

#DATA[:,0:2] = (DATA[:,0:2]  - data_mean)/data_std
#print('DATA normalized:',DATA,'len(DATA) normalized:',len(DATA),'mean dtrain normalized:',DATA.mean(axis=0)[2])
tr_ids = np.random.choice(n*5, n*4, replace = False)



dtrain = DATA[tr_ids,:]
dtest = DATA[-tr_ids,:]

#print('\n','dtrain:',dtrain, 'len(dtrain):',len(dtrain),'mean dtrain:',dtrain.mean(axis=0)[2])
#print('\n','dtest:',dtest, 'len(dtest):',len(dtest),'mean dtest:',dtest.mean(axis=0)[2])

In [ ]:
import math
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import importlib


import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

# define the summary writer
writer = SummaryWriter()
# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

# define the parameters

COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50
TEST_BATCH_SIZE = 100
BATCH_SIZE = 100

l1shape=(2, 5)
l2shape=(5, 5)
l3shape=(5, 5)
l4shape=(5, 5)
l5shape=(5, 5)
l6shape=(5, 5)
layershapes = [l1shape, l2shape, l3shape, l4shape]

# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])
data_shape = (0,2,2,3)

epochs = 30
trtimes  = np.zeros(epochs)
#w_mu = [w_mu1, w_mu2, w_mu3, w_mu4]
#b_mu = [b_mu1, b_mu2, b_mu3, b_mu4]

#w_mu_nodewise = [w_mu1_nodewise,w_mu2_nodewise,w_mu3_nodewise,w_mu4_nodewise]
#b_mu_nodewise = [b_mu1_nodewise,b_mu2_nodewise,b_mu3_nodewise,b_mu4_nodewise]
# make inference on 10 networks

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net3 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(4,4.1),
                                #w_kappa=torch.Tensor(1).uniform_(6,6.1),
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net3.parameters(), lr=0.04)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net3, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        print('max:',net3.weight_mu[1].max())
        print('norm:',torch.norm(net3.weight_mu[1]))

    res = test_ensemble.test_ensemble(net3,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

In [ ]:
import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

epochs = 10

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(4.0,4.1),
                                w_kappa=torch.Tensor(1).uniform_(6.0,6.1),
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4.parameters(), lr=0.14)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    #res = test_ensemble.test_ensemble(net4,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

In [ ]:
GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag.

w_mu = []
for i in range(len(net4.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(net4.weight_mu[i]/torch.norm(net4.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu = []
for i in range(len(net4.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(net4.bias_mu[i]/torch.norm(net4.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho= []
for i in range(len(net4.bias_rho)):
    b_rho.append(net4.bias_rho[i]*GC_intensity)

w_rho= []
for i in range(len(net4.weight_rho)):
    w_rho.append(net4.weight_rho[i]*GC_intensity)

#print('b_rho:',b_rho, '\n','w_rho:',b_mu)

In [ ]:
epochs = 10
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4_GC = FVMF.BayesianNetwork(w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4_GC.parameters(), lr=0.07)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4_GC, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    #res = test_ensemble.test_ensemble(net4_GC,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
#print('This is the loss with the Gradient Capture method/Normalized Initialization Inheritance')

#Ok, så kanskje drit i gradient-capture-method. Men! Husk hva du har vist her nå med kun normalisering av vektene, Du får trent nettet
#Videre, og det fungerer faktisk. I tilfellet hvor du ikke gjør det ender du faktisk ikke bare opp med dårlig ytelse, men
#hele nettet brekker!!!!!!!!!!!

In [ ]:
GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag. GC fungerer rett og slett ikke.

w_mu = []
for i in range(len(net4_GC.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(net4_GC.weight_mu[i]/torch.norm(net4_GC.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu = []
for i in range(len(net4_GC.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(net4_GC.bias_mu[i]/torch.norm(net4_GC.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho= []
for i in range(len(net4_GC.bias_rho)):
    b_rho.append(net4_GC.bias_rho[i]*GC_intensity)

w_rho= []
for i in range(len(net4_GC.weight_rho)):
    w_rho.append(net4_GC.weight_rho[i]*GC_intensity)

#print('b_rho:',b_rho, '\n','w_rho:',b_mu)

In [ ]:
epochs = 10
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4_GC2 = FVMF.BayesianNetwork(w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4_GC2.parameters(), lr=0.035)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4_GC2, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    res = test_ensemble.test_ensemble(net4_GC2,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
print('This is the loss with the Gradient Capture method/Normalized Initialization Inheritance, applied twice')

#Ok, så kanskje drit i gradient-capture-method. Men! Husk hva du har vist her nå med kun normalisering av vektene, Du får trent nettet
#Videre, og det fungerer faktisk. I tilfellet hvor du ikke gjør det ender du faktisk ikke bare opp med dårlig ytelse, men
#hele nettet brekker!!!!!!!!!!!

In [ ]:
RES = 1028 #Ca. Full HD resolution. If you struggle with memory adjust this parameter down. 64 for example already gives a decent view.
enums = 10

DEVICE = torch.device("cuda:1")
Temps = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net4_GC2(Temps[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(torch.max(Temps[:,2]),torch.min(Temps[:,2]))

In [ ]:
plt.title('vMF Uncertainty Plot 42epocsLR0.1 3*10epocs, NII in between.')
#xy = [0,0.2,0.4,0.6,0.8,1]
#z = xy
plot = plt.scatter(Temps[:,0].detach().numpy(), Temps[:,1].detach().numpy(),vmin = 0, vmax = 1.4, s=221000/(RES**2), c=Temps[:,2].detach().numpy(), cmap='RdYlBu', marker='s')
plt.gcf().set_size_inches((14, 12))
plt.colorbar(plot)
plt.show()

In [ ]:
RES = 1028
enums = 10

DEVICE = torch.device("cuda:1")
Temps2 = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps2[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net3(Temps2[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps2[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(probs.shape)

In [ ]:
plt.title('Gaussian Uncertainty Plot')
#xy = [0,0.2,0.4,0.6,0.8,1]
#z = xy
plot2 = plt.scatter(Temps2[:,0].detach().numpy(), Temps2[:,1].detach().numpy(),vmin = 0, vmax = 1.4, s=221000/(RES**2), c=Temps2[:,2].detach().numpy(), cmap='RdYlBu', marker='s')
plt.gcf().set_size_inches((14, 12))
plt.colorbar(plot2)
plt.show()

In [ ]:
RES = 1028
enums = 10

DEVICE = torch.device("cuda:1")
Temps2 = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps2[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net3(Temps2[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps2[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(probs.shape)

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42069)
torch.manual_seed(42069)

PL = [torch.Tensor([1-3, 1-3]),torch.Tensor([5-3, 1-3]),torch.Tensor([1-3, 5-3]),torch.Tensor([5-3, 5-3]),torch.Tensor([3-3, 3-3])]
cov = torch.eye(2)
n = 50

plt.figure(figsize=(10, 10), dpi=500)

DF = torch.zeros((len(PL),n,1,3)) #multiple of 4
for i, MU in enumerate(PL): #enumerate starts from and including 0.
    distrib = torch.distributions.MultivariateNormal(loc=MU, covariance_matrix=cov)
    DATA_ = distrib.sample((n,1))

    DATA  = torch.zeros([n, 1, 3])
    DATA[:,:,:2] = DATA_
    DATA[:,:,2]  = i
    DF[i,:,:,:] = DATA
    
    x = DATA[:,0,0]
    y = DATA[:,0,1]
    plt.plot(x,y,'.',markersize=1.25)
plt.show()

C = int(3*n/4)

#DATA_train = torch.zeros((len(PL)*C,3))
#DATA_test = torch.zeros((len(PL)*(n-C),3))

DATA = DF.reshape(len(PL)*n,3)
#print('DATA:',DATA,'len(DATA):',len(DATA),'mean dtrain:',DATA.mean(axis=0)[2])

#data_mean = DATA.mean(axis=1)[0:2]
#data_std = DATA.std(axis=1)[0:2]

#DATA[:,0:2] = (DATA[:,0:2]  - data_mean)/data_std
#print('DATA normalized:',DATA,'len(DATA) normalized:',len(DATA),'mean dtrain normalized:',DATA.mean(axis=0)[2])
tr_ids = np.random.choice(n*5, n*4, replace = False)



dtrain = DATA[tr_ids,:]
dtest = DATA[-tr_ids,:]

#print('\n','dtrain:',dtrain, 'len(dtrain):',len(dtrain),'mean dtrain:',dtrain.mean(axis=0)[2])
#print('\n','dtest:',dtest, 'len(dtest):',len(dtest),'mean dtest:',dtest.mean(axis=0)[2])

import math
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm, trange
import pandas as pd
import time
import mpmath
import os
import importlib


import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

# define the summary writer
writer = SummaryWriter()
# select the device
DEVICE = torch.device("cuda:1")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
cuda = torch.cuda.set_device(1)

# define the parameters

COND_OPT = False
CLASSES = 5
# TRAIN_EPOCHS = 250
SAMPLES = 1
TEST_SAMPLES = 10
TEMPER = 0.001
TEMPER_PRIOR = 0.001
pepochs = 50
TEST_BATCH_SIZE = 100
BATCH_SIZE = 100

l1shape=(2, 5)
l2shape=(5, 5)
l3shape=(5, 5)
l4shape=(5, 5)
l5shape=(5, 5)
l6shape=(5, 5)
layershapes = [l1shape, l2shape, l3shape, l4shape]

# set prior parameters
PI = 1
SIGMA_1 = torch.cuda.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.cuda.FloatTensor([math.exp(-6)])
data_shape = (0,2,2,3)

epochs = 30
trtimes  = np.zeros(epochs)
#w_mu = [w_mu1, w_mu2, w_mu3, w_mu4]
#b_mu = [b_mu1, b_mu2, b_mu3, b_mu4]

#w_mu_nodewise = [w_mu1_nodewise,w_mu2_nodewise,w_mu3_nodewise,w_mu4_nodewise]
#b_mu_nodewise = [b_mu1_nodewise,b_mu2_nodewise,b_mu3_nodewise,b_mu4_nodewise]
# make inference on 10 networks

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net3 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='Gaussian',
                                #b_kappa=torch.Tensor(1).uniform_(4,4.1),
                                #w_kappa=torch.Tensor(1).uniform_(6,6.1),
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net3.parameters(), lr=0.04)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net3, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        print('max:',net3.weight_mu[1].max())
        print('norm:',torch.norm(net3.weight_mu[1]))

    res = test_ensemble.test_ensemble(net3,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

import test_ensemble
importlib.reload(test_ensemble)
import FVMF
importlib.reload(FVMF)

epochs = 10

for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4 = FVMF.BayesianNetwork(#w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa=torch.Tensor(1).uniform_(4.0,4.1),
                                w_kappa=torch.Tensor(1).uniform_(6.0,6.1),
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4.parameters(), lr=0.14)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    #res = test_ensemble.test_ensemble(net4,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")

GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag.

w_mu = []
for i in range(len(net4.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(net4.weight_mu[i]/torch.norm(net4.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu = []
for i in range(len(net4.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(net4.bias_mu[i]/torch.norm(net4.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho= []
for i in range(len(net4.bias_rho)):
    b_rho.append(net4.bias_rho[i]*GC_intensity)

w_rho= []
for i in range(len(net4.weight_rho)):
    w_rho.append(net4.weight_rho[i]*GC_intensity)

#print('b_rho:',b_rho, '\n','w_rho:',b_mu)

epochs = 10
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4_GC = FVMF.BayesianNetwork(w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4_GC.parameters(), lr=0.07)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4_GC, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    #res = test_ensemble.test_ensemble(net4_GC,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
#print('This is the loss with the Gradient Capture method/Normalized Initialization Inheritance')

#Ok, så kanskje drit i gradient-capture-method. Men! Husk hva du har vist her nå med kun normalisering av vektene, Du får trent nettet
#Videre, og det fungerer faktisk. I tilfellet hvor du ikke gjør det ender du faktisk ikke bare opp med dårlig ytelse, men
#hele nettet brekker!!!!!!!!!!!

GC_intensity = 1

#Kanskje er det en god idee å også arve kappaene, men halvere dem? Hva om vi også kjører på med litt normalizering av mu-ene her?
#Hva med adaptiv kappa per lag? Det er jo bare en kappa per bias-lag. GC fungerer rett og slett ikke.

w_mu = []
for i in range(len(net4_GC.weight_mu)):
    #print('\n','torch.norm(net4.weight_mu[i]):',torch.norm(net4.weight_mu[i]))
    w_mu.append(net4_GC.weight_mu[i]/torch.norm(net4_GC.weight_mu[i]))
    #print('\n','norm w_mu[i]',torch.norm(w_mu[i]))
    
b_mu = []
for i in range(len(net4_GC.bias_mu)):
    #print('\n','torch.norm(net4.bais_mu[i]):',torch.norm(net4.bias_mu[i]))
    b_mu.append(net4_GC.bias_mu[i]/torch.norm(net4_GC.bias_mu[i]))
    #print('\n','norm b_mu[i]',torch.norm(b_mu[i]))
    
b_rho= []
for i in range(len(net4_GC.bias_rho)):
    b_rho.append(net4_GC.bias_rho[i]*GC_intensity)

w_rho= []
for i in range(len(net4_GC.weight_rho)):
    w_rho.append(net4_GC.weight_rho[i]*GC_intensity)

#print('b_rho:',b_rho, '\n','w_rho:',b_mu)

epochs = 10
for i in range(0, 1):
    print(i)
    torch.manual_seed(i)
    net4_GC2 = FVMF.BayesianNetwork(w_mu = w_mu, b_mu = b_mu, 
                                #w_mu = None, b_mu = None,
                                #w_mu = w_mu_nodewise, b_mu = b_mu,
                                layershapes = layershapes,
                                dtrain=dtrain, dtest=dtest,
                                VD='vmf',
                                b_kappa= b_rho, #torch.Tensor(1).uniform_(1.0,3.1), 
                                w_kappa= w_rho, #torch.Tensor(1).uniform_(2.0,4.1), 
                                Temper = 1,classification = 'classification')
    
    #for j,p in enumerate(net2.l1.parameters()):    
    #    p.requires_grad_(False)
    #    
    #for j,p in enumerate(net2.l2.parameters()):
    #    p.requires_grad_(False)
    
    optimizer = optim.Adam(net4_GC2.parameters(), lr=0.035)
    
    
    for epoch in range(epochs):

        trtimes[epoch] = FVMF.train(net4_GC2, dtrain, SAMPLES, optimizer, epoch, i,BATCH_SIZE = 100, shape = data_shape)
        #print('max:',net4.weight_mu[1].max())
        #print('norm:',torch.norm(net4.weight_mu[1]))

    res = test_ensemble.test_ensemble(net4_GC2,dtest,TEST_SAMPLES,TEST_BATCH_SIZE,BATCH_SIZE,CLASSES,DEVICE,shape = data_shape)

    #np.savetxt("soundGmaccuracies_" + str(i) + ".csv", res, delimiter=",")
print('This is the loss with the Gradient Capture method/Normalized Initialization Inheritance, applied twice')

#Ok, så kanskje drit i gradient-capture-method. Men! Husk hva du har vist her nå med kun normalisering av vektene, Du får trent nettet
#Videre, og det fungerer faktisk. I tilfellet hvor du ikke gjør det ender du faktisk ikke bare opp med dårlig ytelse, men
#hele nettet brekker!!!!!!!!!!!

RES = 1028 #Ca. Full HD resolution. If you struggle with memory adjust this parameter down. 64 for example already gives a decent view.
enums = 10

DEVICE = torch.device("cuda:1")
Temps = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net4_GC2(Temps[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(torch.max(Temps[:,2]),torch.min(Temps[:,2]))

plt.title('vMF Uncertainty Plot 42epocsLR0.1 3*10epocs, NII in between.')
#xy = [0,0.2,0.4,0.6,0.8,1]
#z = xy
plot = plt.scatter(Temps[:,0].detach().numpy(), Temps[:,1].detach().numpy(),vmin = 0, vmax = 1.4, s=221000/(RES**2), c=Temps[:,2].detach().numpy(), cmap='RdYlBu', marker='s')
plt.gcf().set_size_inches((14, 12))
plt.colorbar(plot)
plt.show()

RES = 1028
enums = 10

DEVICE = torch.device("cuda:1")
Temps2 = torch.zeros((RES**2,3))
i, j = 0, 0

x = torch.linspace(-5,5,RES)

Temps2[:,:2] = torch.cartesian_prod(x, x)
probs = torch.zeros((1,5)).to(DEVICE)

for j in range(enums):
    probs = probs + torch.exp(net3(Temps2[:,0:2])) #This is done enums times, each one will be different due to stochastic.
probs = probs/enums

Temps2[:,2] = -torch.sum(probs * torch.log(probs),dim = 1)
print(probs.shape)